# Chapter 9: Spatial data management - vector formats

GeoPandas is an open source project to make working with geospatial data in Python easier. GeoPandas extends the datatypes used by *pandas* to allow spatial operations on geometric types.

## Install *geopandas*

The following Python packages are required to be installed:
 - geopandas
 - descartes (for visualization)
 - mapclassify (for classification of data)
 - rtree (spatial indexing of data)

### Anaconda - Platform independent

If you have Anaconda installed, open the *Anaconda Prompt* and type in:
```
conda update --all
conda install -c conda forge geopandas descartes mapclassify rtree
```

*Note:* updating the currently installed packages to their most recent version can be required to avoid dependency issues.  
*Note:* we install from the *conda-forge* channel, as it contains more recent versions of these packages compared to the *default* channel of Anaconda.

### Python Package Installer (pip) - Linux

If you have standalone Python3 and Jupyter Notebook install on Linux, open a command prompt / terminal and type in:
```
pip3 install geopandas descartes mapclassify rtree
```
For the *rtree* Python package you must also install the *libspatialindex-dev* system package, which will require administrative priviliges:
```
sudo apt-get install libspatialindex-dev
```

### Python Package Installer (pip) - Windows

The installation of these packages is much more complicated with *pip* on Windows, because several library binaries must be installed separately or compiled from source. (E.g. the *geopandas* package highly depends on the *GDAL* library.)  
An easier approach is to install these packages from [Python binary wheel files](https://www.lfd.uci.edu/~gohlke/pythonlibs/).

Due to its complexity these options are only recommended for advanced Python users and it is **strongly advised to use Anaconda on Windows**.

## How to use *geopandas*?

The geopandas package is also a module which you can simply import. It is usually aliased with the `gpd` abbreviation.
```python
import geopandas as gpd
```

## Read spatial data

Geopandas can read many vector-based spatial data format including Shapefiles, GeoJSON files and much more. Only the `read_file()` function has to be called.
The result is a geopandas dataframe, a *GeoDataFrame*.

Read the `data/ne_10m_admin_0_countries.shp` shapefile located in the `data` folder. This dataset contains both scalar and spatial data of the countries all over the world.  
Source: [Natural Earth](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

countries_gdf = gpd.read_file('data/ne_10m_admin_0_countries.shp')
display(countries_gdf)

*Note:* observe the `geometry` column (the last one), which contains the geometry of the row in a [*well-known text* (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) format.

## Basic usage of *GeoDataFrames*

Since this *GeoDataFrame* has quite a number of columns, some of them are hidden by the display. Let's list all the columns:

In [ ]:
print(countries_gdf.columns)

With a lot of columns it can be useful to select only a few columns to make the displayed results more human-readable.
This can be done by in a similar way when selecting a single *Series* from a *DataFrame*, but now we shall define a list of *Series* to select.  
*Remark:* this makes a copy of the dataframe.

In [ ]:
countries_gdf = countries_gdf[['NAME', 'POP_EST', 'POP_YEAR', 'GDP_MD_EST', 'GDP_YEAR', 'REGION_UN', 'geometry']]
display(countries_gdf)

Geopandas extends the capabilties of the pandas library, which means we can use all what we have learned with pandas.

Let's sort the *GeoDataFrame* by the name of the countries:

In [ ]:
display(countries_gdf.sort_values(by='NAME'))

Filter the dataframe to contain only the European countries:

In [ ]:
condition = countries_gdf['REGION_UN'] == 'Europe'
europe_gdf = countries_gdf[condition]
display(europe_gdf)

Sort the European countries by their population in a descending order:

In [ ]:
display(europe_gdf.sort_values(by = 'POP_EST', ascending = False))

## Map making

Geopandas provides a high-level interface to the *matplotlib* library for making maps. Mapping shapes is as easy as using the `plot()` method on a *GeoDataFrame* (or *GeoSeries*).

In [ ]:
countries_gdf.plot(figsize=[20,10])
plt.show()

The `plot()` function call on a *GeoDataFrame* (or a regular pandas *DataFrame*) will return an axis configuration object, which we can use to further customize our plot (map in this case). E.g. we can hide the axes with the `set_axis_off()` function:

In [ ]:
ax = countries_gdf.plot(figsize=[20,10])
ax.set_axis_off()
plt.show()

### Choropleth maps

Geopandas makes it easy to create so called *choropleth maps* (maps where the color of each shape is based on the value of an associated variable). Simply use the `plot()` method with the `column` argument set to the column whose values you want used to assign colors.

In [ ]:
countries_gdf.plot(column='POP_EST', figsize=[20,10])
plt.show()

Add a legend to the map.

In [ ]:
countries_gdf.plot(column='POP_EST', legend=True, figsize=[20,10])
plt.show()

We can choose from various available color maps. A complete list can be found on the [matplotlib website](https://matplotlib.org/tutorials/colors/colormaps.html).

In [ ]:
countries_gdf.plot(column='GDP_MD_EST', legend=True, cmap='YlOrRd', figsize=[20,10])
plt.show()

The way color maps are scaled can also be manipulated with the `scheme` option (the *mapclassify* Python library must be installed).

A full list of schemes are available on the project's [GitHub page](https://github.com/pysal/mapclassify) and some examples of result on the [package's website](
https://pysal.org/mapclassify/index.html).


In [ ]:
countries_gdf.plot(column='GDP_MD_EST', legend=True, cmap='YlOrRd', figsize=[20,10], scheme='quantiles')
plt.show()

## Multiple layers

We can easily combine the data of multiple *GeoDataFrames* and even visualize them as multiple layers with geopandas.

Open and read a second data source defined in the `data/World_Cities.shp` shapefile, containing scalar and spatial data about major cities all around the world.  
Source: [ArcGIS](https://hub.arcgis.com/datasets/6996f03a1b364dbab4008d99380370ed_0)

In [ ]:
cities_gdf = gpd.read_file('data/World_Cities.shp')
display(cities_gdf)

Reduce the number of columns, by selecting only the now important ones:

In [ ]:
cities_gdf = cities_gdf[['CITY_NAME', 'CNTRY_NAME', 'STATUS', 'POP', 'geometry']]
display(cities_gdf)

Plot the cities:

In [ ]:
cities_gdf.plot(color='red', markersize=3, figsize=[20,10])
plt.show()

Verify whether both datasets use the same coordinate reference system:

In [ ]:
print(cities_gdf.crs)
print(countries_gdf.crs)

Would be they different, geopandas would also be capable to transform one of the dataframes to the other CRS:
```python
cities_gdf = cities_gdf.to_crs(countries_gdf.crs)
```

Create a combined visualization of multiple layers, by simply calling the `plot()` method on all *GeoDataFrames*, but drawing them on the same axis object.

In [ ]:
base = countries_gdf.plot(color='white', edgecolor='black', figsize=[20, 10])
cities_gdf.plot(ax=base, color='red', markersize=3)
plt.show()

## Clipping operation

Geopandas offers a coordinate indexer (`cx`), which can be used to select only the records which geomtry overlaps with the selected region.

Let's select and plot the countries in the northern hemisphere.

In [ ]:
northern_gdf = countries_gdf.cx[:, 0:]
northern_gdf.plot(figsize=[20, 10])
plt.show()

*Note:* with this approach countries overlapping both the northern and southern hemispheres are not clipped.

We can perform real clipping with the `clip()` function of geopandas. As a showcase let's clip the countries and country parts inside the bounding box of Europe; defined with the following polygon (given in WKT format):  
`POLYGON ((-10 35, 40 35, 40 70, -10, 70, -10, 35))`.

Geopandas uses the [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) library in the background to represent and manipulate vector data. Therefore first define a regular pandas *DataFrame* named `europe_df`, where the *Coordinates* column will contain a polygon defined with *Shapely*.

In [ ]:
import pandas as pd
from shapely.geometry import Polygon

europe_df = pd.DataFrame({
    'Name': ['Europe'],
    'Coordinates': [Polygon([(-10, 35), (40, 35), (40, 70), (-10, 70), (-10, 35)])]
    # the polygon is defined as a closed line
})
display(europe_df)

Now our *GeoDataFrame* can be constructed from the *DataFrame* stored in `europe_df`, by defining which *Series* (column) contains the geometries and the CRS. (Use the CRS of the countries dataset.)

In [ ]:
europe_gdf = gpd.GeoDataFrame(europe_df, geometry='Coordinates', crs=countries_gdf.crs)
display(gdf)

Finally, we can perform the clipping operation between the *GeoDataFrames*:

In [ ]:
clipped_gdf = gpd.clip(countries_gdf, europe_gdf)
clipped_gdf.plot(figsize=[10, 10])
plt.show()

## Spatial operations

The spatial join (`sjoin()`) function of *geopandas* performs a spatial intersection check between the records of one or two *GeoDataFrames*. (The *rtree* package must be installed for spatial indexing support.)

Let's match the countries and cities based on their spatial location:

In [ ]:
display(gpd.sjoin(countries_gdf, cities_gdf))

Limit the number of columns displayed to get an output easier to interpret:

In [ ]:
display(gpd.sjoin(countries_gdf, cities_gdf)[['NAME', 'CITY_NAME']])

Select the cities inside Hungary for a quick verification of the results:

In [ ]:
condition = countries_gdf['NAME'] == 'Hungary'
hungary_gdf = countries_gdf[condition]
display(gpd.sjoin(hungary_gdf, cities_gdf)[['NAME', 'CITY_NAME']])

Perform a spatial intersection check between the dataframe containing only Hungary (`hungary_gdf`) and the dataframe containing all countries (`countries_gdf`). The result shall be the neighbouring countries of Hungary.

In [ ]:
display(gpd.sjoin(hungary_gdf, countries_gdf)[['NAME_left', 'NAME_right']])

*Remark:* the `NAME` column was renamed to `NAME_left` and `NAME_right` automatically, since column names must be unique.

## Writing spatial data

*GeoDataFrames* can be easily persisted with the `to_file()` function. As when reading files, various file formats are supported again.

In [ ]:
clipped_gdf.to_file('09_clipped.shp')
#clipped_gdf.to_file('09_clipped2.geojson', driver='GeoJSON')

---

## Summary exercises on vector data management

Beside the `countries_gdf` *GeoDataFrame*, read the `data/ne_10m_rivers_lake_centerlines.shp` shapefile located in the `data` folder. This dataset contains both scalar and spatial data of the larger rivers and lakes around the world.  
Source: [Natural Earth](https://www.naturalearthdata.com/downloads/10m-physical-vectors/)

In [ ]:
rivers_gdf = gpd.read_file('data/ne_10m_rivers_lake_centerlines.shp')
display(rivers_gdf)

## Exercise 1

Viualize the country boundaries and the river/lake layers on the same map. (Rivers and lakes shall be blue.)

In [ ]:
base = countries_gdf.plot(color='white', edgecolor='black', figsize=[20, 10])
rivers_gdf.plot(ax=base, color='blue')
plt.show()

## Exercise 2

Visualize only Hungary (on any preferred country) and the rivers flowing through it.

In [ ]:
hungary_gdf = countries_gdf[countries_gdf['NAME'] == 'Hungary']
hungary_rivers = gpd.sjoin(rivers_gdf, hungary_gdf)

base = hungary_gdf.plot(color='white', edgecolor='black', figsize=[20, 10])
hungary_rivers.plot(ax=base, color='blue')
plt.show()

With clipping to country boundaries:

In [ ]:
hungary_rivers = gpd.clip(hungary_rivers, hungary_gdf)

base = hungary_gdf.plot(color='white', edgecolor='black', figsize=[20, 10])
hungary_rivers.plot(ax=base, color='blue')
plt.show()

## Exercise 3

Determine for the river *Danube* (on any major river) that which countries it flows through.

*Hint: the river might consist of multiple line segments in the river dataset, but you can filter all of them by e.g. the `name_en` field.*

In [ ]:
danube_gdf = rivers_gdf[rivers_gdf['name_en'] == 'Danube']
display(danube_gdf)

danube_countries = gpd.sjoin(danube_gdf, countries_gdf)
display(danube_countries[['NAME']])